In [1]:
%matplotlib inline

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx

import time
import numpy as np

In [54]:
# for parallelisation
from joblib import Parallel, delayed

import multiprocessing as mp
from functools import partial

In [51]:
def compute_shortest_path_distances_parallel(graph):
    num_cores = multiprocessing.cpu_count()
    print(num_cores, 'cores used')

    shortest_paths = Parallel(n_jobs=num_cores, batch_size=10)(delayed(nx.single_source_shortest_path_length)(graph, n) for n in graph)

    distances = []
    for _dict in shortest_paths:
        for key, value in _dict.items():
            distances.append(value)

    return distances

In [57]:
def compute_shortest_path_distances_parallel_mp(graph):
    num_cores = mp.cpu_count()
    print(num_cores, 'cores used')
    
    pool = mp.Pool(processes=num_cores)

    parallel_function = partial(nx.single_source_shortest_path_length, graph)
    sources = [source for source in graph]
    shortest_paths = pool.map(parallel_function, sources)

    pool.close()
    pool.join()

    distances = []
    for _dict in shortest_paths:
        for key, value in _dict.items():
            distances.append(value)

    return distances

In [2]:
%%time

path_to_file = './data/wiki-Vote/wiki-Vote.txt'
G_dir = nx.read_edgelist(path_to_file, comments='#', delimiter='\t', nodetype=int, create_using=nx.DiGraph())
G_undir = nx.read_edgelist(path_to_file, comments='#', delimiter='\t', nodetype=int)

CPU times: user 718 ms, sys: 28.4 ms, total: 746 ms
Wall time: 750 ms


In [18]:
%%time
LSCC = max(nx.strongly_connected_component_subgraphs(G_dir), key=len)

CPU times: user 17.8 s, sys: 38.6 ms, total: 17.9 s
Wall time: 18 s


In [19]:
%%time
LWCC = max(nx.weakly_connected_component_subgraphs(G_dir), key=len)
LWCC = nx.to_undirected(LWCC)

CPU times: user 507 ms, sys: 16.3 ms, total: 523 ms
Wall time: 531 ms


In [17]:
%%time
print('LSCC edges: \t', LSCC.number_of_edges())
print('LSCC nodes: \t', LSCC.number_of_nodes())

print('LWCC edges: \t',LWCC.number_of_edges())
print('LWCC nodes: \t',LWCC.number_of_nodes())

LSCC edges: 	 39456
LSCC nodes: 	 1300
LWCC edges: 	 103663
LWCC nodes: 	 7066
CPU times: user 33.1 ms, sys: 2.86 ms, total: 35.9 ms
Wall time: 42.6 ms


In [58]:
%%time
length_arr = compute_shortest_path_distances_parallel_mp(LSCC)

4 cores used
CPU times: user 587 ms, sys: 115 ms, total: 702 ms
Wall time: 9.29 s


In [53]:
%%time
length_dir = dict(nx.all_pairs_shortest_path_length(LSCC))

CPU times: user 18 s, sys: 74.6 ms, total: 18.1 s
Wall time: 18.2 s


In [6]:
%%time
LSCC_nodes = LSCC.nodes()
dist_dir = []
for node_i in LSCC_nodes:
    for node_j in LSCC_nodes:
        if node_i != node_j:
            dist_dir.append(length_dir[node_i][node_j])
        
print('median distance:\t',np.percentile(dist_dir,50))
print('mean distance:\t',np.mean(dist_dir))
print('diameter:\t',np.max(dist_dir))
print('effective diameter:\t',np.percentile(dist_dir,90))

median distance:	 3.0
mean distance:	 2.87928288032
diameter:	 9
effective diameter:	 4.0
CPU times: user 847 ms, sys: 27.1 ms, total: 874 ms
Wall time: 884 ms


In [59]:
%%time
length_arr_LWCC = compute_shortest_path_distances_parallel_mp(LWCC)

4 cores used
CPU times: user 10.4 s, sys: 2.65 s, total: 13 s
Wall time: 9min 28s


In [42]:
%%time
length_undir = dict(nx.all_pairs_shortest_path_length(LWCC))

CPU times: user 19min 23s, sys: 5.68 s, total: 19min 29s
Wall time: 19min 45s


In [43]:
%%time
dist_undir = []
LWCC_nodes = LWCC.nodes()
for node_i in LWCC_nodes:
    for node_j in LWCC_nodes:
        if node_i != node_j:
            dist_undir.append(length_undir[node_i][node_j])
        
print('median distance:\t',np.percentile(dist_undir,50))
print('mean distance:\t',np.mean(dist_undir))
print('diameter:\t',np.max(dist_undir))
print('effective diameter:\t',np.percentile(dist_undir,90))

median distance:	 3.0
mean distance:	 3.24750999023
diameter:	 7
effective diameter:	 4.0
CPU times: user 22 s, sys: 1.21 s, total: 23.2 s
Wall time: 23.6 s


# compute distance between two nodes for all nodes manually --> it is slower!

In [9]:
%%time
start = time.time()
dist_dir = []
count = 0
LSCC_nodes = LSCC.nodes()
for i in LSCC_nodes:
    for j in LSCC_nodes:
        dist_dir.append(nx.shortest_path_length(LSCC,i,j))
        if(count%100000 == 0):
            print(np.round(count/10**6,1), 'mio \t', round(time.time() - start,1), '\t sec since start')
        count = count + 1

0.0 mio 	 0.0 	 sec since start
0.1 mio 	 2.5 	 sec since start
0.2 mio 	 4.8 	 sec since start
0.3 mio 	 7.3 	 sec since start
0.4 mio 	 9.6 	 sec since start
0.5 mio 	 12.3 	 sec since start
0.6 mio 	 15.1 	 sec since start
0.7 mio 	 18.4 	 sec since start
0.8 mio 	 21.6 	 sec since start
0.9 mio 	 25.0 	 sec since start
1.0 mio 	 28.0 	 sec since start
1.1 mio 	 31.2 	 sec since start
1.2 mio 	 34.2 	 sec since start
1.3 mio 	 37.3 	 sec since start
1.4 mio 	 40.4 	 sec since start
1.5 mio 	 43.5 	 sec since start
1.6 mio 	 46.8 	 sec since start
CPU times: user 48.3 s, sys: 396 ms, total: 48.7 s
Wall time: 49.7 s


In [10]:
%%time
start = time.time()
dist_undir = []
count = 0
for i in LWCC.nodes:
    for j in LWCC.nodes:
        if nx.has_path(LWCC,i,j):
            dist_dir.append(nx.shortest_path_length(LWCC,i,j))
        if(count%100000 == 0):
            print(round(time.time() - start,2), '\t seconds since start')
        count = count + 1

0.0 	 seconds since start
8.17 	 seconds since start
16.33 	 seconds since start
23.44 	 seconds since start
29.93 	 seconds since start
37.67 	 seconds since start
45.08 	 seconds since start
50.09 	 seconds since start
56.74 	 seconds since start
63.71 	 seconds since start
69.71 	 seconds since start
78.03 	 seconds since start
84.88 	 seconds since start
91.66 	 seconds since start
98.13 	 seconds since start
103.83 	 seconds since start
110.07 	 seconds since start
115.99 	 seconds since start
121.29 	 seconds since start
127.02 	 seconds since start
131.91 	 seconds since start
138.6 	 seconds since start
143.82 	 seconds since start
150.12 	 seconds since start
157.49 	 seconds since start
164.05 	 seconds since start
171.01 	 seconds since start
176.73 	 seconds since start
182.64 	 seconds since start
190.41 	 seconds since start
195.73 	 seconds since start
202.43 	 seconds since start
207.81 	 seconds since start
213.27 	 seconds since start
218.23 	 seconds since start


KeyboardInterrupt: 

In [4]:
%%time
start = time.time()
dist_dir = []
count = 0
for i in G_dir.nodes:
    for j in G_dir.nodes:
        if nx.has_path(G_dir,i,j):
            dist_dir.append(nx.shortest_path_length(G_dir,i,j))
        if(count%100000 == 0):
            print(round(time.time() - start,2), '\t seconds since start')
        count = count + 1

0.0 	 seconds since start
3.52 	 seconds since start
5.41 	 seconds since start
9.43 	 seconds since start
12.55 	 seconds since start
14.87 	 seconds since start
17.53 	 seconds since start
19.41 	 seconds since start
21.79 	 seconds since start
24.49 	 seconds since start
26.72 	 seconds since start
28.43 	 seconds since start
30.45 	 seconds since start
32.05 	 seconds since start
35.24 	 seconds since start
37.6 	 seconds since start
39.11 	 seconds since start
41.72 	 seconds since start
43.99 	 seconds since start
46.4 	 seconds since start
48.14 	 seconds since start
50.28 	 seconds since start
51.91 	 seconds since start
53.16 	 seconds since start
56.11 	 seconds since start
58.21 	 seconds since start
60.35 	 seconds since start
62.63 	 seconds since start
64.26 	 seconds since start
66.03 	 seconds since start
69.49 	 seconds since start
72.7 	 seconds since start
75.03 	 seconds since start
76.99 	 seconds since start
79.63 	 seconds since start
82.95 	 seconds since start


In [11]:
print('median distance: ',np.percentile(dist_dir,50))
print('mean distance: ',np.mean(dist_dir))
print('diameter: ',np.max(dist_dir))
print('effective diameter: ',np.percentile(dist_dir,90))

median distance:  3.0
mean distance:  3.33902266947
diameter:  10
effective diameter:  4.0


In [14]:
nx.shortest_path_length(LWCC,30,)

0